# Pytorch

## Instalaciones

In [ ]:
!pip install torch
!pip install torchvision

## Importar dependencias

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Definir Hiperparámetros

In [ ]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 1e-3

## Descargando la base de datos mnist

In [ ]:
train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 14685407.83it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 446568.39it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4115649.05it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3434384.13it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## Leyendo la data

In [ ]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

## Definir modelo

In [ ]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

## Instancia del modelo

In [ ]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

## Compilación

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

## Entrenamiento

In [ ]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))


Epoca [1/5], Step [100/600], Loss: 0.4559
Epoca [1/5], Step [200/600], Loss: 0.4267
Epoca [1/5], Step [300/600], Loss: 0.3127
Epoca [1/5], Step [400/600], Loss: 0.2185
Epoca [1/5], Step [500/600], Loss: 0.2348
Epoca [1/5], Step [600/600], Loss: 0.2218
Epoca [2/5], Step [100/600], Loss: 0.2211
Epoca [2/5], Step [200/600], Loss: 0.2011
Epoca [2/5], Step [300/600], Loss: 0.1639
Epoca [2/5], Step [400/600], Loss: 0.1687
Epoca [2/5], Step [500/600], Loss: 0.2342
Epoca [2/5], Step [600/600], Loss: 0.1260
Epoca [3/5], Step [100/600], Loss: 0.1867
Epoca [3/5], Step [200/600], Loss: 0.1310
Epoca [3/5], Step [300/600], Loss: 0.1031
Epoca [3/5], Step [400/600], Loss: 0.1041
Epoca [3/5], Step [500/600], Loss: 0.1167
Epoca [3/5], Step [600/600], Loss: 0.0915
Epoca [4/5], Step [100/600], Loss: 0.1160
Epoca [4/5], Step [200/600], Loss: 0.0785
Epoca [4/5], Step [300/600], Loss: 0.1132
Epoca [4/5], Step [400/600], Loss: 0.1525
Epoca [4/5], Step [500/600], Loss: 0.1708
Epoca [4/5], Step [600/600], Loss:

In [ ]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output, 1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy: %.3f %%' % (100 * correct / (total + 1)))

Accuracy: 96.540 %
